In [ ]:
!git clone https://github.com/google-research/bert

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/igcar_ps/ /mydrive

In [ ]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
#import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")



from bert import modeling, optimization, tokenization
#from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()

log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s : \
    %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    
    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        with open('/content/adc.json', 'r') as f:
          auth_info = json.load(f)
        
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
        
       
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

In [ ]:
BUCKET_PATH = "gs://ayushjain1144-bucket/"
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = BUCKET_PATH + BERT_MODEL
print('****** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR)) 
#!ls BERT_PRETRAINED_DIR

In [ ]:
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt.*'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'

In [ ]:
QA_DATASET = BUCKET_PATH + 'qa_dataset'
QA_TRAIN_FILE = QA_DATASET + '/train-v1.1.json'
QA_PREDICT_FILE = QA_DATASET + '/dev-v1.1.json'
QA_EVALUATE = QA_DATASET + '/evaluate-v1.1.py'


In [ ]:
OUTPUT_DIR = BUCKET_PATH + 'output-dir/'

In [ ]:
PRED_JSON = OUTPUT_DIR + '/predictions.json'

In [ ]:
XARGS_CMD = ("python3 bert/run_squad.py "
            "--vocab_file={} "
            "--bert_config_file={} "
            "--init_checkpoint={} "
            "--do_train=True "
            "--train_file={} "
            "--do_predict=True "
            "--predict_file={} "
            "--train_batch_size=12 "
            "--learning_rate=3e-5 "
            "--num_train_epochs=2.0 "
            "--max_seq_length=256 "
            "--doc_stride=128 "
            "--output_dir={} "
            )
XARGS_CMD = XARGS_CMD.format(VOCAB_FILE, 
                            BERT_CONFIG,
                            INIT_CHECKPOINT,
                            QA_TRAIN_FILE,
                            QA_PREDICT_FILE,
                            OUTPUT_DIR)
                           
print(XARGS_CMD)

python3 bert/run_squad.py --vocab_file=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=gs://ayushjain1144-bucket/uncased_L-12_H-768_A-12/bert_model.ckpt --do_train=True --train_file=gs://ayushjain1144-bucket/qa_dataset/train-v1.1.json --do_predict=True --predict_file=gs://ayushjain1144-bucket/qa_dataset/dev-v1.1.json --train_batch_size=12 --learning_rate=3e-5 --num_train_epochs=2.0 --max_seq_length=256 --doc_stride=128 --output_dir=gs://ayushjain1144-bucket/output-dir/ 


In [ ]:
!$XARGS_CMD

In [ ]:
!python3 /mydrive/qa_dataset/evaluate-v1.1.py /mydrive/qa_dataset/dev-v1.1.json output-dir_predictions.json